In [19]:
import os
import random
import pandas as pd
from transformers import pipeline
import numpy as np
from tqdm.notebook import tqdm

In [20]:
import_df = pd.read_csv("bertTesting.csv")
test_files = []
for tf in list(import_df["0"]):
    test_files.append(tf[:-4])
test_files = test_files[:1]

In [21]:
def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    output = list(range(start_index, start_index + token_len))
    if output[-1] >= len(full_text.split()):
        output = list(range(start_index, start_index + token_len-1))
    return output

def predict_and_format(fileName, predict_df, predictAcc_df):
    data = ""
    with open('train/' + fileName + ".txt", 'r') as file:
        data = file.read().replace('\n', '')
        data = data.replace("Â\xa0", '')
    
    predicts = token_classifier(data)

    for p in predicts:
        word_Indices_Array = calc_word_indices(data, p["start"], p["end"])
        word_Indices_String = " ".join(str(x) for x in word_Indices_Array)
        word_class = p["entity_group"]
        word_id = fileName
        predictAcc_df.loc[len(predictAcc_df.index)] = [word_id, word_class, word_Indices_String]
        if len(word_Indices_Array) >= 3:
            predict_df.loc[len(predict_df.index)] = [word_id, word_class, word_Indices_String]

In [22]:
# code by ROB MULLA
# source: https://www.kaggle.com/robikscube/student-writing-competition-twitch#Competition-Metric-Code

def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(" "))
    set_gt = set(row.predictionstring_gt.split(" "))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter / len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp_micro(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition

    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = (
        gt_df[["id", "discourse_type", "predictionstring"]]
        .reset_index(drop=True)
        .copy()
    )
    pred_df = pred_df[["id", "class", "predictionstring"]].reset_index(drop=True).copy()
    pred_df["pred_id"] = pred_df.index
    gt_df["gt_id"] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(
        gt_df,
        left_on=["id", "class"],
        right_on=["id", "discourse_type"],
        how="outer",
        suffixes=("_pred", "_gt"),
    )
    joined["predictionstring_gt"] = joined["predictionstring_gt"].fillna(" ")
    joined["predictionstring_pred"] = joined["predictionstring_pred"].fillna(" ")

    joined["overlaps"] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5,
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined["overlap1"] = joined["overlaps"].apply(lambda x: eval(str(x))[0])
    joined["overlap2"] = joined["overlaps"].apply(lambda x: eval(str(x))[1])

    joined["potential_TP"] = (joined["overlap1"] >= 0.5) & (joined["overlap2"] >= 0.5)
    joined["max_overlap"] = joined[["overlap1", "overlap2"]].max(axis=1)
    tp_pred_ids = (
        joined.query("potential_TP")
        .sort_values("max_overlap", ascending=False)
        .groupby(["id", "predictionstring_gt"])
        .first()["pred_id"]
        .values
    )

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined["pred_id"].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query("potential_TP")["gt_id"].unique()
    unmatched_gt_ids = [c for c in joined["gt_id"].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    # calc microf1
    my_f1_score = TP / (TP + 0.5 * (FP + FN))
    return my_f1_score


def score_feedback_comp(pred_df, gt_df, return_class_scores=False):
    class_scores = {}
    pred_df = pred_df[["id", "class", "predictionstring"]].reset_index(drop=True).copy()
    for discourse_type, gt_subset in gt_df.groupby("discourse_type"):
        pred_subset = (
            pred_df.loc[pred_df["class"] == discourse_type]
            .reset_index(drop=True)
            .copy()
        )
        class_score = score_feedback_comp_micro(pred_subset, gt_subset)
        class_scores[discourse_type] = class_score
    f1 = np.mean([v for v in class_scores.values()])
    if return_class_scores:
        return f1, class_scores
    return f1

In [23]:
def make_predictions(test_files):
    dict = {'id':[],
        'class': [],
        'predictionstring':[]}
    predict_df = pd.DataFrame(dict)
    predictAcc_df = pd.DataFrame(dict)
    error = 0
    success = 0
    for file in tqdm(test_files):
        try:
            predict_and_format(file, predict_df, predictAcc_df)
            success = success + 1
        except:
            error = error + 1
    print(success)
    print(error)
    return predict_df, predictAcc_df

In [24]:
def ground_truth(test_files):
    full_df = pd.read_csv("train.csv")
    full_df = full_df.set_index("id")
    train_df = full_df.loc[test_files]
    train_df = train_df.reset_index()
    return train_df
    

In [25]:
bert_checkpoint = "brad1141/Bert_v5"
longformer_checkpoint = "brad1141/Longformer_v5"
gpt2_checkpoint = "brad1141/GPT2_v5"

#change model to the model api that you wish to use
#the three models mentioned in the paper are listed above
model = longformer_checkpoint
# model = "OldData_bertv2"
token_classifier = pipeline(
    "token-classification", model=model, aggregation_strategy="simple", tokenizer=model
)

In [26]:
results_df, resultsAcc_df = make_predictions(test_files)
gt_df = ground_truth(test_files)

  0%|          | 0/1 [00:00<?, ?it/s]

1
0


In [27]:
f1_score, cs = score_feedback_comp(results_df, gt_df, return_class_scores=True)
print(model)
print(f1_score)
print(cs)

brad1141/Longformer_v5
0.5666666666666667
{'Claim': 0.0, 'Concluding Statement': 1.0, 'Counterclaim': 1.0, 'Evidence': 0.4, 'Position': 0.0, 'Rebuttal': 1.0}


In [33]:
avg_acc = []
labels = []
predictions = []
tokenDict  = {"LABEL_0": 'Lead', 
    "LABEL_1": 'Position', 
    "LABEL_2": 'Evidence', 
    "LABEL_3": 'Claim', 
    "LABEL_4": 'Concluding Statement', 
    "LABEL_5": 'Counterclaim', 
    "LABEL_6": 'Rebuttal', 
    "LABEL_7": 'No Class',
}
for index, row in resultsAcc_df.iterrows():
    wordCount = len(row["predictionstring"].split())
    for w in range(wordCount):
        predictions.append(row["class"])
for index, row in gt_df.iterrows():
    wordCount = len(row["predictionstring"].split())
    for w in range(wordCount):
        labels.append(row["discourse_type"])

In [34]:
print(len(predictions))
print(len(labels))
predictions = predictions[:len(labels)]

261
251


In [29]:
from datasets import load_metric

metric = load_metric("seqeval")

In [35]:
metric.compute(predictions=[predictions], references=[labels])

C:\Users\Brad\anaconda3\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'ead': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'ebuttal': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'laim': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'oncluding Statement': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 1},
 'osition': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ounterclaim': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'vidence': {'precision': 0.36619718309859156,
  'recall': 0.3170731707317073,
  'f1': 0.33986928104575165,
  'number': 82},
 'overall_precision': 0.2857142857142857,
 'overall_recall': 0.2988505747126437,
 'overall_f1': 0.2921348314606742,
 'overall_accuracy': 0.46613545816733065}